In [145]:
#URL: https://www.kaggle.com/datasets/lantian773030/pokemonclassification

# 기존 프로젝트: 켓몬 149마리의 이미지 데이터 셋을 흑백 이미지로 변환하여 
# 분류 및 MLP,CNN 방식으로 학습하여 47%의 정확도를 얻음

#트레이닝 데이터 80%
#테스트 데이터 20%

#캐글에 올라온 예제를 일부 변경하여 성능 개선 시도
#합성곱 레이어를 늘려서 좀 더 복잡한 패턴을 만든다.
#optimizer 파라미터를 조절하면서 최적을 찾는다.
#batch 파라미터를 조절하면서 최적을 찾는다.
#epoch 파라미터를 조절하면서 최적을 찾는다.



In [146]:
import os #os 모듈 가져오기
import random 
import numpy as np
import pandas as pd
from tqdm import tqdm #notebook 자동으로 진행상태를 시각적으로 보여주는 라이브러리
import torch
import torch.nn as nn #신경망을 생성하고 학습을 도움, 잘 디자인된 모듈과 클래스들을 제공
import torch.nn.functional as F 
from torch.utils.data import random_split 
     #데이터 세트를 분리하기 위해 torch.utils.data에서 random_split 함수를 포함시킵
from torch.utils.data import DataLoader, Dataset, Subset
from torch.utils.data import random_split, SubsetRandomSampler 
     #트레이닝 및 테스트 데이터 지정
from torchvision import datasets, transforms, models #터치비전에서 데이터 가져오기
from torchvision.datasets import ImageFolder 
     #대용량 이미지 데이터를 데이터셋을 관리하기 편한 모듈
from torchvision.transforms import ToTensor #이미지를 텐서로 변환
from torchvision.utils import make_grid #그리드 만들기
from pytorch_lightning import LightningModule
     #lightning module은 trainer와 model이 상호작용할 수 있게 해주는 구현체
from pytorch_lightning import Trainer
import pytorch_lightning as pl
import matplotlib.pyplot as plt 
     #모듈의 각각의 함수를 사용해서 간편하게 그래프를 만들고 변화를 줄 수 있음
%matplotlib inline
from sklearn.model_selection import train_test_split #테스트, 검증 데이터 구분
from sklearn.metrics import classification_report 
     #sklearn.metrics에서 라이브러리 불러오기
from PIL import Image #이미지처리 라이브러리

In [147]:
#이미지 조정
transform=transforms.Compose([
        transforms.RandomRotation(10),      # 회전 +/- 10 degrees
        transforms.RandomHorizontalFlip(),  # 반전 50% of images
        transforms.Resize(224),             # 리사이즈 짧은쪽을 224 pixel로
        transforms.CenterCrop(224),         # 긴쪽을 중심에서 224 pixel로
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
])

In [148]:
dataset0=datasets.ImageFolder(root="./data/PokemonData/",transform=None)
        #지정된 경로로 데이터 불러오기
class_names=dataset0.classes
print(class_names) #데이터 클래스 이름 프린트
print(len(class_names))

['Abra', 'Aerodactyl', 'Alakazam', 'Alolan Sandslash', 'Arbok', 'Arcanine', 'Articuno', 'Beedrill', 'Bellsprout', 'Blastoise', 'Bulbasaur', 'Butterfree', 'Caterpie', 'Chansey', 'Charizard', 'Charmander', 'Charmeleon', 'Clefable', 'Clefairy', 'Cloyster', 'Cubone', 'Dewgong', 'Diglett', 'Ditto', 'Dodrio', 'Doduo', 'Dragonair', 'Dragonite', 'Dratini', 'Drowzee', 'Dugtrio', 'Eevee', 'Ekans', 'Electabuzz', 'Electrode', 'Exeggcute', 'Exeggutor', 'Farfetchd', 'Fearow', 'Flareon', 'Gastly', 'Gengar', 'Geodude', 'Gloom', 'Golbat', 'Goldeen', 'Golduck', 'Golem', 'Graveler', 'Grimer', 'Growlithe', 'Gyarados', 'Haunter', 'Hitmonchan', 'Hitmonlee', 'Horsea', 'Hypno', 'Ivysaur', 'Jigglypuff', 'Jolteon', 'Jynx', 'Kabuto', 'Kabutops', 'Kadabra', 'Kakuna', 'Kangaskhan', 'Kingler', 'Koffing', 'Krabby', 'Lapras', 'Lickitung', 'Machamp', 'Machoke', 'Machop', 'Magikarp', 'Magmar', 'Magnemite', 'Magneton', 'Mankey', 'Marowak', 'Meowth', 'Metapod', 'Mew', 'Mewtwo', 'Moltres', 'MrMime', 'Muk', 'Nidoking', 'Ni

In [149]:
class DataModule(pl.LightningDataModule): 
        #파이토치 라이트닝의 데이터모듈 클래스를 활용해서 트레이닝을 위한 데이터 준비와 구성
    
    def __init__(self, transform=transform, batch_size=32): 
        #배치 파라미터 설정(적정한 파라미터를 찾기 위해 여러번 진행함)
        super().__init__()
        self.root_dir = "./data/PokemonData/"
        self.transform = transform
        self.batch_size = batch_size

    def setup(self, stage=None):
        dataset = datasets.ImageFolder(root=self.root_dir, transform=self.transform)
        n_data = len(dataset)
        n_train = int(0.8 * n_data) #트레이닝 데이터와 테스트 데이터를 랜덤으로 8대2 비율로 적용
        n_test = n_data - n_train

        train_dataset, test_dataset = torch.utils.data.random_split(dataset, [n_train, n_test])

        self.train_dataset = DataLoader(train_dataset, batch_size=self.batch_size, shuffle=True)
        self.test_dataset = DataLoader(test_dataset, batch_size=self.batch_size)

    def train_dataloader(self):
        return self.train_dataset

    def test_dataloader(self):
        return self.test_dataset



In [150]:
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, random_split
from torchvision.datasets import ImageFolder
import pytorch_lightning as pl

class BetterCNN(pl.LightningModule):
    def __init__(self):
        super(BetterCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 3, 1) 
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.conv3 = nn.Conv2d(64, 128, 3, 1)
        self.fc1 = nn.Linear(128 * 12 * 12, 512)
        self.fc2 = nn.Linear(512, 128)
        self.fc3 = nn.Linear(128, len(class_names))
        self.dropout = nn.Dropout(0.5) # drop out레이어로 전체 레이어 추가 후 오버피팅을 줄인다.
        self.pool = nn.MaxPool2d(2, 2)

    def forward(self, X):
        X = self.pool(F.relu(self.conv1(X)))
        X = self.pool(F.relu(self.conv2(X)))
        X = self.pool(F.relu(self.conv3(X)))
        X = X.view(-1, 128 * 12 * 12)#파라미터 변경(-1, 16 * 54 * 54)
        X = F.relu(self.fc1(X))
        X = self.dropout(X)
        X = F.relu(self.fc2(X))
        X = self.dropout(X)
        X = self.fc3(X)
        return F.log_softmax(X, dim=1)

    def configure_optimizers(self):#아담을 옵티마이저로
        optimizer = torch.optim.Adam(self.parameters(), lr=0.0001)
        return optimizer

    def training_step(self, train_batch, batch_idx):
        X, y = train_batch
        y_hat = self(X)
        loss = F.cross_entropy(y_hat, y)
        pred = y_hat.argmax(dim=1, keepdim=True)
        acc = pred.eq(y.view_as(pred)).sum().item() / y.shape[0]
        self.log("train_loss", loss)
        self.log("train_acc", acc)
        return loss

    def validation_step(self, val_batch, batch_idx):
        X, y = val_batch
        y_hat = self(X)
        loss = F.cross_entropy(y_hat, y)
        pred = y_hat.argmax(dim=1, keepdim=True)
        acc = pred.eq(y.view_as(pred)).sum().item() / y.shape[0]
        self.log("val_loss", loss)
        self.log("val_acc", acc)

    def test_step(self, test_batch, batch_idx):
        X, y = test_batch
        y_hat = self(X)
        loss = F.cross_entropy(y_hat, y)
        pred = y_hat.argmax(dim=1, keepdim=True)
        acc = pred.eq(y.view_as(pred)).sum().item() / y.shape[0]
        self.log("test_loss", loss)
        self.log("test_acc", acc)

In [ ]:
if __name__ == '__main__':
    datamodule = DataModule()
    datamodule.setup()
    model = ConvolutionalNetwork()
    trainer = pl.Trainer(max_epochs=50)#epoch 조절
    trainer.fit(model, datamodule)
    datamodule.setup(stage='test')
    test_loader = datamodule.test_dataloader()
    trainer.test(dataloaders=test_loader)

GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name  | Type   | Params
---------------------------------
0 | conv1 | Conv2d | 168   
1 | conv2 | Conv2d | 880   
2 | fc1   | Linear | 5.6 M 
3 | fc2   | Linear | 10.2 K
4 | fc3   | Linear | 1.7 K 
5 | fc4   | Linear | 3.1 K 
---------------------------------
5.6 M     Trainable params
0         Non-trainable params
5.6 M     Total params
22.460    Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

In [ ]:
#파이토치 데이터로더로부터 이미지 시각화

for images, labels in datamodule.train_dataloader(): #데이터 로딩 및 이미지 시각화 구성 설정
    break
im=make_grid(images,nrow=16)

plt.figure(figsize=(12,12))
plt.imshow(np.transpose(im.numpy(),(1,2,0)))

#노멀랑이징 후 이미지 그리드 만들기
inv_normalize=transforms.Normalize(mean=[-0.485/0.229,-0.456/0.224,-0.406/0.225],
                                   std=[1/0.229,1/0.224,1/0.225])
im=inv_normalize(im)

plt.figure(figsize=(12,12))
plt.imshow(np.transpose(im.numpy(),(1,2,0)))

In [ ]:
#slearn.metrics에서 classification_report기능을 활용하여 파이토치 모델 평가

device = torch.device("cpu") #cpu를 디바이스로 설정

model.eval() #모델 평가 모드로 설정
y_true=[]
y_pred=[]
with torch.no_grad(): 
    for test_data in datamodule.test_dataloader():#테스트 데이터배치 반복
        test_images, test_labels = test_data[0].to(device), test_data[1].to(device)
        #테스트 데이터 계산을 위해서 (CPU/GPU)로 전송
        pred = model(test_images).argmax(dim=1)
        #테스트 이미지들을 모델을 거쳐서 예측 값 구하기
        #argmax를 이용해서 가능한 최고값 클래스를 선택
        for i in range(len(pred)):#반복 실행
            y_true.append(test_labels[i].item())
            y_pred.append(pred[i].item())

print(classification_report(y_true,y_pred,target_names=class_names,digits=4)) #출력

In [ ]:
#optimizer, batch, epoch 등의 파라미터들을 조정해가며 성능체크를 진행하였다. 
#결과적으로 기존 프로젝트 정확도 47%에서 60~70%의 정확도로 개선할 수 있었다. 